In [ ]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.network.models import (
    NetworkSecurityGroup,
    SecurityRule,
    NetworkInterface,
    NetworkInterfaceIPConfiguration,
    Subnet,
    PublicIPAddress,
)
from azure.mgmt.resource import ResourceManagementClient

In [ ]:
subscription_id = ""
resource_group = ""
resource_group_vnet = ''
location = ""


tamanio_instancia = ""
ssh_key = ""
nombre_base = ''

nsg_name = 'nsg-driver'

sistema_operativo = {
            "publisher": "",
            "offer": "",
            "sku": "",
            "version": ""
        }



# Parámetros que vendrían como argumento
vnet_name = ""
subnet_name = ""
public_ip_name = nombre_base + "-ip"
nic_name = nombre_base + "-nic"
vm_name = nombre_base 
username = "azureuser"


In [ ]:
cred = DefaultAzureCredential()

# Clientes
resource_client = ResourceManagementClient(cred, subscription_id)
network_client = NetworkManagementClient(cred, subscription_id)
compute_client = ComputeManagementClient(cred, subscription_id)

In [ ]:
# Crear NSG
nsg_params = NetworkSecurityGroup(location=location)
nsg = network_client.network_security_groups.begin_create_or_update(
    resource_group,
    nsg_name,
    nsg_params
).result()

print(f"NSG creado: {nsg.id}")

In [ ]:
# Regla SSH
# ssh_rule = SecurityRule(
#     protocol="Tcp",
#     source_port_range="*",
#     destination_port_range="22",
#     source_address_prefix="*",
#     destination_address_prefix="*",
#     access="Allow",
#     direction="Inbound",
#     priority=100,
#     name="AllowSSH"
# )

# network_client.security_rules.begin_create_or_update(
#     resource_group,
#     nsg_name,
#     "AllowSSH",
#     ssh_rule
# ).result()

# Regla HTTP
http_rule = SecurityRule(
    protocol="Tcp",
    source_port_range="*",
    destination_port_range="80",
    source_address_prefix="*",
    destination_address_prefix="*",
    access="Allow",
    direction="Inbound",
    priority=200,
    name="AllowHTTP"
)

network_client.security_rules.begin_create_or_update(
    resource_group,
    nsg_name,
    "AllowHTTP",
    http_rule
).result()

print("Reglas agregadas al NSG")

In [ ]:
subnet = network_client.subnets.get(resource_group_vnet, vnet_name, subnet_name)

In [ ]:
public_ip = network_client.public_ip_addresses.begin_create_or_update(
    resource_group,
    public_ip_name,
    {
        "location": location,
        "sku": {"name": "Standard"},
        "public_ip_allocation_method": "Static"
    }
).result()

In [ ]:
nic_params = NetworkInterface(
    location=location,
    ip_configurations=[NetworkInterfaceIPConfiguration(
        name="default",
        subnet=Subnet(id=subnet.id),
        public_ip_address=PublicIPAddress(id=public_ip.id),
    )],
    network_security_group=nsg 
)

nic = network_client.network_interfaces.begin_create_or_update(
    resource_group,
    nic_name,
    nic_params
).result()

In [ ]:
ssh_key = compute_client.ssh_public_keys.get(
    resource_group_name=resource_group,
    ssh_public_key_name=ssh_key
)

ssh_key_data = ssh_key.public_key

In [ ]:
# ⚡ 4. Crear la VM
vm_parameters = {
    "location": location,
    "hardware_profile": {"vm_size": tamanio_instancia},
    "storage_profile": {"image_reference": sistema_operativo},
    "os_profile": {
        "computer_name": vm_name,
        "admin_username": username,
        "linux_configuration": {
            "disable_password_authentication": True,
            "ssh": {
                "public_keys": [
                    {
                        "path": f"/home/{username}/.ssh/authorized_keys",
                        "key_data": ssh_key_data,
                    }
                ]
            },
        },
    },
    "network_profile": {"network_interfaces": [{"id": nic.id, "primary": True}]},
    "os_disk": {
        "name": nombre_base + "disk",
        "create_option": "FromImage",
        "disk_size_gb": 32,  # 🔹 Aquí defines el tamaño del OS Disk en GB
        "managed_disk": {"storage_account_type": "StandardSSD_LRS"},
    },
}

In [ ]:
compute_client.virtual_machines.begin_create_or_update(
    resource_group,
    vm_name,
    vm_parameters
).result()

print(f"✅ VM '{vm_name}' creada con IP estática {public_ip.ip_address}")